In [11]:
from transformers import pipeline
import pandas as pd
import itertools

modelName = "google/muril-base-cased"

topResults = 100
unmasker = pipeline('fill-mask', model=modelName,top_k=topResults)

config.json: 100%|██████████| 411/411 [00:00<00:00, 1.18MB/s]
pytorch_model.bin: 100%|██████████| 953M/953M [02:41<00:00, 5.89MB/s] 
Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
tokenizer_config.json: 100%|██████████| 206/206 [00:00<00:00, 141kB/s]
vocab.txt: 100%|██████████| 3.16M/3.16M [00:00<00:00, 5.72MB/s]
special_tokens_map.json: 100%|█████████

# Data Cleaning

In [12]:
filePath_template = "../nlp-fairness-for-india-main/templates.tsv"
filePath_religion_idterms = "../nlp-fairness-for-india-main/religion_idterms.tsv"
filePath_region_idterms = "../nlp-fairness-for-india-main/region_idterms.tsv"
filePath_Caste_idterms = "../nlp-fairness-for-india-main/caste_idterms.tsv"
filePath_gender_idterms = "../nlp-fairness-for-india-main/gender_idterms.tsv"
humanDataset_region = "../nlp_fairness_India_datasets/region_stereotypes.tsv"
humanDataset_religion = "../nlp_fairness_India_datasets/religion_stereotypes.tsv"

def getTemplateArray(filePath):
    df_placeolders = pd.read_csv(filePath,sep='\t')
    df_placeolders = df_placeolders.iloc[:,1:]
    df_placeolders = df_placeolders.replace('@','[MASK]',regex=True)
    semiTemplates = df_placeolders.values.tolist()
    templates = [item for sublist in semiTemplates for item in sublist]
    return templates

def getIdentityArray(filePath):
    df_religion = pd.read_csv(filePath,sep='\t')
    religion_list = df_religion.values.tolist()
    religion_idTerms = [item for sublist in religion_list for item in sublist]
    return religion_idTerms

def replaceSlotWithWord(sentences, word):
    return word, [sentence.replace('[SLOT]', word) for sentence in sentences]


def extract_common_tuples(file1_path,file2_path,social_axis):
    file1_df = pd.read_csv(file1_path,sep='\t')
    file2_df = pd.read_csv(file2_path,sep='\t')

    merged_df = pd.merge(file1_df, file2_df, on=[social_axis, 'StereoType'], how='inner')
    merged_df.drop_duplicates(inplace=True)

    return merged_df

def crossProduct(list1, list2):
    cross_product = list(itertools.product(list1, list2))
    return cross_product


templates = getTemplateArray(filePath_template)

# Religion

In [17]:
religionIDTerms = getIdentityArray(filePath_religion_idterms)
religionTuples = []

for religion in religionIDTerms:
    currReligion , modifiedTemplates = replaceSlotWithWord(templates,religion)
    for template in modifiedTemplates:
        result = unmasker(template)
        for res in result:
            religionTuples.append((currReligion,res['token_str']))

df_religionTuples = pd.DataFrame(religionTuples, columns=['Religion', 'StereoType'])
output_file_path = "religionTuples_MLM.tsv"
df_religionTuples.to_csv(output_file_path, sep='\t', index=False)

print("Data saved successfully.")

Data saved successfully.


# Region

In [18]:
regionIDTerms = getIdentityArray(filePath_region_idterms)
regionTuples = []

for region in regionIDTerms:
    currRegion , modifiedTemplates = replaceSlotWithWord(templates,region)
    for template in modifiedTemplates:
        result = unmasker(template)
        for res in result:
            regionTuples.append((currRegion,res['token_str']))

df_regionTuples = pd.DataFrame(regionTuples, columns=['Region', 'StereoType'])
output_file_path = "regionTuples_MLM.tsv"
df_regionTuples.to_csv(output_file_path, sep='\t', index=False)

print("Data saved successfully.")

Data saved successfully.
